# TP1: Algoritmos de búsqueda en Torre de Hanoi.



### 1.¿Cuáles son los PEAS de este problema? (Performance, Environment, Actuators, Sensors)
### Performance

La performance esta dada por haber llegado a la solucion, es decir, colocar a todos los discos en la varilla objetivo en el menor tiempo posible. En este caso como el costo de cada nodo es el mismo (1 para cada movimiento), la solucion optima es la que menos costo tiene.

### Enviroment
El ambiente esta dado por los discos y las varillas.

### Actuator
Asumiendo que el agente es un robot el actuador puede ser una mano robotica que realiza los movimientos de los discos.

### Sensor
En la misma linea que el punto anterior, podria tratarse de una camara y sensores de posicion que posea el robot y para detectar donde estan los discos y donde colocarlos (asi como donde esta situado el brazo, segun el angulo).

### 2.¿Cuáles son las propiedades del entorno de trabajo?

1. Totalmente observable: Una vez que estan seteadas las varillas y los discos, lo unico que puede cambiar es su posicion respetando las reglas del juego.
2. Deterministico: dado un estado en el que se encuentra el agente, se sabe con certeza los siguientes estados posibles. Esto ocurre para todas las transiciones de estados.
3. Secuencial: ya que cada decision presente puede afectar decisiones futuras.
4. Estatico: ya que el entorno no cambia (y tampoco el rendimiento) mientras el agente decide su proxima accion.
5. Discreto: ya que la cantidad de acciones que peude elegir es finita.
6. Agente individual: no hay otros agentes ejecutando acciones sobre los discos o las varas.


### 3. En el contexto de este problema, establezca cuáles son los: estado, espacio de estados, árbol de búsqueda, nodo de búsqueda, objetivo, acción y frontera.

Estado: Un estado es una "foto" a cualquier combinacion de discos correctamente colocada entre las 3 varillas.
Por ejemplo, lo siguiente seria un estado:

![state_example]("./imgs/state_example.png")

Espacio de estados: esta determinado por la cantidad de estados que hay. Si tomamos
k como el numero de varillas y w como el numero de discos, hay k^w = 3^5 = 243 estados. 
Arbol de busqueda: Es el arbol que se forma cuando se realiza la busqueda sobre el grafo de espacio de estados.

Nodo de busqueda: Cada nodo representa un estado en el espacio de estados y las aristas en el arbol de busqueda corresponde a las acciones (que nos llevarian al siguiente nodo)
Objetivo: El estado con 5 discos en la tercer varilla. 
Accion: Mover un disco a otra varilla
Frontera: Dado un nodo explorado, son los nodos (inexplorados) siguientes a los cuales las acciones (desde el nodo explorado) nos podrian llevar.

### 5. Complejidad algoritmica:
El algoritmo implementado es el de A*.
Vas expandiendo nodos para encontrar la solucion: 
En el pero de los casos podes tomar 3 decisiones.
3^(niveles hasta q encuentra)

explorando todos los posibles estados. 
3^5 = 243


#### Calculo de f de A*
Veamos el calculo de f en la priority queue:
```python
good_order = {tuple(range(5, 5 - (i + 1), -1)) for i in range(5)}

def h(new_node):
    rod_3 = new_node.state.rods[2]
    return -len(rod_3) if tuple(rod_3) in good_order else 0

def f(new_node):
    return new_node.path_cost + h(new_node)
```
Debido a que good_order es un set con los discos en el orden correcto y debido a que esta implementado 
con una hash table, consultar si un elemento esta es O(1). Luego, h(n) es de orden O(1).
g(n) es O(1) ya que es simplemente acceder al costo, que esta almacenado en el objeto nodo.
f(n) por ende, es de O(1) tambien.

### Priority Queue complejidad

Veamos las caracteristicas de la priority queue de heapq:
heappush: O(log(n))
heappop: O(log(n))

Sabiendo que el codigo de aima es:
```python
    def append(self, item):
        """Insert item at its correct position."""
        heapq.heappush(self.heap, (self.f(item), item))
    def pop(self):
        """Pop and return the item (with min or max f(x) value)
        depending on the order."""
        if self.heap:
            return heapq.heappop(self.heap)[1]
        else:
            raise Exception('Trying to pop from empty PriorityQueue.')
```
Por ende las lineas
```python
    node = pq.pop()
```
y 
```python
    pq.append(next_node)
```
Son de orden O(log(n)).
Donde n es la cantidad de nodos en la priority queue.


Ahora, sabiendo que agregamos una heuristica consistente, estamos recortando varios posibles caminos malos y nos tendemos a extender por la solucion optima.

```python  
while len(pq) != 0:
    node = pq.pop() # O(log(n)) 
    explored.add(node.state) # O(1) ya que es un set y no van a haber colisiones
    if problem.goal_test(node.state): # check de goal test es O(1) tambien
        last_node = node
        print("Encontramos la solución")
        break
    for next_node in node.expand(problem): # esto en peor caso se extiende b veces
            if next_node.state not in explored: # este chequeo es O(1) ya que explored es un set
                pq.append(next_node) # esto es O(log(n))
```
h = 0
log(n) (pop) + b*log(n) (append) 
h = 1 
b*log(n) (pop) + b*(b*log(n)) (append)
h = 2 
b^2*log(n)

...

log(n)*(b^h) (pops) + (b^h) * log(n) (append)

O(log(n)) * (b^h))

n = b^h 
log(b^h) = log(b)*h

=> O(b^h).

### Calculo de effective branching factor.
En el algoritmo de A* se suele caracterizar la complejidad mediante un effective branching factor llamado b*.
Esto es meramente una estimacion para saber cuanto puede reducir la complejidad el hecho de agregar una cola de prioridad y heuristica.
Como utilizamos A* con una heuristica consistente, por ende debe llegar a la solucion optima.
El peor caso de complejidad es haber recorrido el arbol completo, es decir, O(b^h).
Sabemos que la solucion optima requiere de 2^k -1 movimentos siendo k el numero de discos.
k = 5 =>  h = 31.
La cantidad de nodos es 256, por ende 
N + 1 = (b*)^(h+1) - 1
log2(N+2) = log2(b*)*(h+1)
log2(N+2)/(h+1) = log2(b*)
b* = 1.189496356
En donde b* seria nuestro effective branching factor. 

Complejidad tanto en tiempo como espacio es: O((b*)^h )





### 6.A nivel implementación, ¿qué tiempo y memoria ocupa el algoritmo? (Se recomienda correr 10 veces y calcular promedio y desvío estándar de las métricas).

In [21]:
from aima import PriorityQueue
from hanoi_states import StatesHanoi, ProblemHanoi
from tree_hanoi import NodeHanoi
last_node = None

In [22]:
%%timeit
initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)

problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]
explored = set()

good_order = {tuple(range(5, 5 - (i + 1), -1)) for i in range(5)}

def h(new_node):
    rod_3 = new_node.state.rods[2]
    return -len(rod_3) if tuple(rod_3) in good_order else 0


def f(new_node):
    return new_node.path_cost + h(new_node)


pq = PriorityQueue(order='min', f=f)
pq.append(NodeHanoi(problem.initial))

while len(pq) != 0:
    node = pq.pop()
    explored.add(node.state)
    if problem.goal_test(node.state):
        last_node = node
        # print("Encontramos la solución")
        break
    for next_node in node.expand(problem):
        if next_node.state not in explored:
            pq.append(next_node)

18.1 ms ± 349 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
import tracemalloc

# Para medir memoria consumida (usamos el pico de memoria)
tracemalloc.start()

initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)

problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]
explored = set()

good_order = {tuple(range(5, 5 - (i + 1), -1)) for i in range(5)}

def h(new_node):
    rod_3 = new_node.state.rods[2]
    return -len(rod_3) if tuple(rod_3) in good_order else 0


def f(new_node):
    return new_node.path_cost + h(new_node)


pq = PriorityQueue(order='min', f=f)
pq.append(NodeHanoi(problem.initial))

while len(pq) != 0:
    node = pq.pop()
    explored.add(node.state)
    if problem.goal_test(node.state):
        last_node = node
        # print("Encontramos la solución")
        break
    for next_node in node.expand(problem):
        if next_node.state not in explored:
            pq.append(next_node)
            
_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

print(f"Maxima memoria ocupada: {round(memory_peak, 2)} [MB]", )

Maxima memoria ocupada: 0.22 [MB]


In [24]:
import tracemalloc

# Para medir memoria consumida (usamos el pico de memoria)
tracemalloc.start()

# Inicializaos el problema
initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]  # Creamos una cola FIFO con el nodo inicial

explored = set()  # Este set nos permite ver si ya exploramos un estado para evitar repetir indefinidamente

# Mientras que la cola no este vacia
while len(frontier) != 0:
    node = frontier.pop()  # Extraemos el primer nodo de la cola
    
    # Agregamos nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
    explored.add(node.state)
    
    if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
        last_node = node
        break
    
    # Agregamos a la cola todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        # Solo si no fue explorado
        if next_node.state not in explored:
            frontier.insert(0, next_node)
            
_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

print(f"Maxima memoria ocupada: {round(memory_peak, 2)} [MB]", )

Maxima memoria ocupada: 1.61 [MB]


### 7. Si la solución óptima es movimientos con k igual al número de discos. Qué tan lejos está la solución del algoritmo implementado de esta solución óptima (se recomienda correr al menos 10 veces y usar el promedio de trayecto usado).

Como se trata de A* con una heuristica consistente la solucion implementada converge a la solucion optima.
 



In [26]:
last_node.generate_solution_for_simulator()

In [27]:
print(last_node)

<Node HanoiState:  |  | 5 4 3 2 1>
